In [1]:
modify script to get rid of join

SyntaxError: invalid syntax (174206673.py, line 1)

In [1]:
import os,sys, arcpy
import pandas as pd
import numpy as np
from arcpy.sa import* 

arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

gdb = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb'
SFHA = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\FFRMS_Processing\Working_Data\Eff_data.gdb\S_FLD_HAZ_AR'
Depth_Grid = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\Depth_Grid.gdb\FL_12097_17N_03m_WSEL_Con'
FVA_00 = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\Depth_Grid.gdb\FL_12097_17N_03m_FVA00_Con'
FVA_01 = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\Depth_Grid.gdb\FL_12097_17N_03m_FVA01_Con'
FVA_02 = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\Depth_Grid.gdb\FL_12097_17N_03m_FVA02_Con'
FVA_03 = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\Depth_Grid.gdb\FL_12097_17N_03m_FVA03_Con'



sfha = arcpy.GetParameterAsText(0)

datasets = []
fc_all = []
ds_paths = []
fc_dict = {}
fc_dictx = {}
surface = [Depth_Grid,FVA_00,FVA_01,FVA_02,FVA_03]
zones = {'1pct':Depth_Grid,'FVA00':FVA_00,'FVA01':FVA_01,'FVA02':FVA_02,'FVA03':FVA_03}

def env(x):
    arcpy.env.workspace = x
    return arcpy.env.workspace

def paths(x,y):
    path = os.path.join(x,y)
    return path

env(gdb)

feature_datasets = arcpy.ListDatasets()
for dataset in feature_datasets:
    datasets.append(dataset)

for ds in datasets:
    ds_path = paths(gdb,ds)
    ds_paths.append(ds_path)
print(ds_paths)


for dsp in ds_paths:
    env(dsp)
    fc_list = arcpy.ListFeatureClasses()
    print(len(fc_list))
    parse_dsp = dsp.split('\\')
    dsp_name = parse_dsp[-1]
    dsp_final = dsp_name.replace("_"," ")

    arcpy.SetProgressor("Step","Exporting Tables for Dataset: {0}...".format(dsp_final),0,len(fc_list))
    for fc in fc_list:
        fc_desc = arcpy.Describe(fc)
        fc_name = fc_desc.name
        fc_path = paths(dsp,fc_name)
        spa = fc_desc.SpatialReference
        print(fc_path)
        geo_type = fc_desc.shapeType
        FID_Field = "FID_"+fc_name
        int_list = [SFHA,fc]
        geo_result_join = fc_path+"_int_join"
        geo_result_int = fc_path+"_int"
        geo_result = fc_path+'_final'
            
        arcpy.SetProgressorLabel("Exporting {0} data...".format(fc_name.replace("_"," ")))

        def exp_geo_line_poly(x,y,z):

            arcpy.analysis.PairwiseIntersect(
                in_features=x,
                out_feature_class=y,
                join_attributes="ALL",
                output_type="INPUT")

            print('FLD Zone Intersect Completed')

            int_fl = arcpy.management.MakeFeatureLayer(y,'int_fl')

            fields = ['Z_Max_1pct','Z_Mean_1pct',
                      'Z_Max_FVA00','Z_Mean_FVA00',
                      'Z_Max_FVA01','Z_Mean_FVA01',
                      'Z_Max_FVA02','Z_Mean_FVA02',
                      'Z_Max_FVA03','Z_Mean_FVA03']
            

            
            arcpy.management.AddFields(
                in_table=int_fl,
                field_description=[[fields[0],'DOUBLE',fields[0]],
                                [fields[1],'DOUBLE',fields[1]],
                                [fields[2],'DOUBLE',fields[2]],
                                [fields[3],'DOUBLE',fields[3]],
                                [fields[4],'DOUBLE',fields[4]],
                                [fields[5],'DOUBLE',fields[5]],
                                [fields[6],'DOUBLE',fields[6]],
                                [fields[7],'DOUBLE',fields[7]],
                                [fields[8],'DOUBLE',fields[8]],
                                [fields[9],'DOUBLE',fields[9]]])
            
            # build table names and execute zonal stats
            for item in surface:
                for key, value in zones.items():
                    if value == item:
                        tbl = paths(gdb,fc_name+'_'+key+'_Zonal')

                        #Execute Zonal Stats for each surface
                        ZonalStatisticsAsTable(
                        in_zone_data=int_fl,
                        zone_field='OBJECTID',
                        in_value_raster=item,
                        out_table=tbl,
                        ignore_nodata='DATA',
                        statistics_type='ALL',
                        percentile_values=90)
            
                        print(f'zonal {key} calculated')

                        field_zonal_dict_max = {}
                        field_zonal_dict_mean = {}

                        with arcpy.da.SearchCursor(tbl,['OBJECTID_1','MAX','MEAN']) as cursor1:
                            for row1 in cursor1:
                                OID = row1[0]
                                zmax = row1[1]
                                zmean = row1[2]

                                field_zonal_dict_max[OID] = zmax
                                field_zonal_dict_mean[OID] = zmean

                        update_fields = [f'Z_Max_{key}',f'Z_Mean_{key}','OBJECTID']

                        print(f'{key} mean and max dictionaries built')

                        for key, value in field_zonal_dict_max.items():
                            with arcpy.da.UpdateCursor(int_fl,update_fields) as cursor2:
                                for row2 in cursor2:
                                    if row2[2] == key:
                                        row2[0] = value 
                                        cursor2.updateRow(row2)

                        for key, value in field_zonal_dict_mean.items():
                            with arcpy.da.UpdateCursor(int_fl,update_fields) as cursor3:
                                for row3 in cursor3:
                                    if row3[2] == key:
                                        row3[1] = value 
                                        cursor3.updateRow(row3)

                        print(field_zonal_dict_max)

            # Copy feature layer to gdb
            product = arcpy.management.CopyFeatures(
                in_features=int_fl,
                out_feature_class=z)
            
            return product

        exp_geo_line_poly(int_list,geo_result_int,geo_result)

['C:\\Project_Files\\GIS\\Library\\Dev_Apps\\HMP_Tools\\HMP_Tools\\Temp_Processing.gdb\\Natural_cultural_historical_resource']
7
C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb\Natural_cultural_historical_resource\St_Cloud_Park_Building
FLD Zone Intersect Completed
zonal 1pct calculated
1pct mean and max dictionaries built
{1: None, 2: None, 3: None, 4: None, 5: None, 6: None, 7: None, 8: None, 9: None, 10: None, 11: None, 12: None, 13: None, 14: 0.10000000149011612, 15: 0.10000000149011612, 16: 0.10000000149011612, 17: 0.3149681091308594, 18: 0.17174911499023438, 19: 1.1974220275878906, 20: 0.10000000149011612, 21: 0.10000000149011612, 22: 0.10000000149011612, 23: 0.10000000149011612, 24: None, 25: None, 26: None, 27: None, 28: None, 29: None, 30: None, 31: None, 32: None, 33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 48: None, 49: None, 50: None, 51: N

exporting results micro analysis

In [ ]:
var = 'electric_production_supply_facilities_substations_FVA03_Zonal_MAX'
print(len(var))

if len(var) > 64:
    # Trim the string to 62 characters and add '2' to the end
    new_string = var[:63] + '2'
    # If you want to maintain the last 3 characters ('MAX'), adjust accordingly
else:
    # If the string is already 64 characters or less, keep it as it is
    new_string = var

print(new_string)


65
electric_production_supply_facilities_substations_FVA03_Zonal_M2


In [ ]:
import os,sys, arcpy
import pandas as pd
import numpy as np
from arcpy.sa import* 

arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

gdb = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb'
SFHA = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\FFRMS_Processing\Working_Data\Eff_data.gdb\S_FLD_HAZ_AR'
Depth_Grid = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\Depth_Grid.gdb\FL_12097_17N_03m_WSEL_Con'
FVA_00 = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\Depth_Grid.gdb\FL_12097_17N_03m_FVA00_Con'
FVA_01 = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\Depth_Grid.gdb\FL_12097_17N_03m_FVA01_Con'
FVA_02 = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\Depth_Grid.gdb\FL_12097_17N_03m_FVA02_Con'
FVA_03 = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\Depth_Grid.gdb\FL_12097_17N_03m_FVA03_Con'
csv_point = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Folder\JPA_Sensitivity_Analysis_Point.csv'
csv_ln_pt = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Folder\JPA_Sensitivity_Analysis_Line_Poly.csv'
csv_asset = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Folder\JPA_Sensitivity_Analysis_Asset_List.csv'


sfha = arcpy.GetParameterAsText(0)

datasets = []
fc_all = []
ds_paths = []
fc_dict = {}
fc_dictx = {}
surface = [Depth_Grid,FVA_00,FVA_01,FVA_02,FVA_03]
zones = {'1_pct':Depth_Grid,'FVA00':FVA_00,'FVA01':FVA_01,'FVA02':FVA_02,'FVA03':FVA_03}

def env(x):
    arcpy.env.workspace = x
    return arcpy.env.workspace

def paths(x,y):
    path = os.path.join(x,y)
    return path

env(gdb)

feature_datasets = arcpy.ListDatasets()
for dataset in feature_datasets:
    datasets.append(dataset)

for ds in datasets:
    ds_path = paths(gdb,ds)
    ds_paths.append(ds_path)
print(ds_paths)


for dsp in ds_paths:
    env(dsp)
    fc_list = arcpy.ListFeatureClasses()
    print(len(fc_list))
    parse_dsp = dsp.split('\\')
    dsp_name = parse_dsp[-1]
    dsp_final = dsp_name.replace("_"," ")

    arcpy.SetProgressor("Step","Exporting Tables for Dataset: {0}...".format(dsp_final),0,len(fc_list))
    for fc in fc_list:
        fc_desc = arcpy.Describe(fc)
        fc_name = fc_desc.name
        fc_path = paths(dsp,fc_name)
        spa = fc_desc.SpatialReference
        print(fc_path)
        geo_type = fc_desc.shapeType
        FID_Field = "FID_"+fc_name
        int_list = [SFHA,fc]
        geo_result_join = fc_path+"_int_join"
        geo_result_int = fc_path+"_int"
        geo_result = fc_path+'_final'
            
        arcpy.SetProgressorLabel("Exporting {0} data...".format(fc_name.replace("_"," ")))

        def exp_geo_line_poly(v,w,x,y,z):

            arcpy.analysis.PairwiseIntersect(
                in_features=v,
                out_feature_class=y,
                join_attributes="ALL",
                output_type="INPUT")

            print('FLD Zone Intersect Completed')

            arcpy.management.CalculateField(
                in_table=y,
                field='FID',
                expression='!{0}!'.format(w),
                expression_type='PYTHON3'
            )

            int_fl = arcpy.management.MakeFeatureLayer(y,'int_fl')

            # build table names and execute zonal stats
            for item in surface:
                for key, value in zones.items():
                    if value == item:
                        tbl = paths(gdb,fc_name+'_'+key+'_Zonal')

                        #Execute Zonal Stats for each surface
                        ZonalStatisticsAsTable(
                        in_zone_data=int_fl,
                        zone_field='OBJECTID',
                        in_value_raster=item,
                        out_table=tbl,
                        ignore_nodata='DATA',
                        statistics_type='ALL',
                        percentile_values=90)
            
                        print(f'zonal {key} calculated')

                        #Execute Join for each zonal table
                        arcpy.management.AddJoin(
                            in_layer_or_view=int_fl,
                            in_field='OBJECTID',
                            join_table=tbl,
                            join_field='OBJECTID',
                            join_type='KEEP_ALL')
                        print(f'zonal {key} join executed')

            # Copy feature layer to gdb
            arcpy.management.CopyFeatures(
                in_features=int_fl,
                out_feature_class=x)

            # taregt joined z fields
            zonal_z_Fields=[
                fc_name+'_1_pct_Zonal_MAX',fc_name+'_1_pct_Zonal_MEAN',
                fc_name+'_FVA00_Zonal_MAX',fc_name+'_FVA00_Zonal_MEAN',
                fc_name+'_FVA01_Zonal_MAX',fc_name+'_FVA01_Zonal_MEAN',
                fc_name+'_FVA02_Zonal_MAX',fc_name+'_FVA02_Zonal_MEAN',
                fc_name+'_FVA03_Zonal_MAX',fc_name+'_FVA03_Zonal_MEAN']
            
            updated_zonal_z_Fields = []

            for item in zonal_z_Fields:
                if len(item) >= 64:
                    if item.split('_')[-1] == 'MAX':
                        updated_item = item[:63] + '2'
                    elif item.split('_')[-1] == 'MEAN':
                        updated_item = item[:63] + '3'
                    updated_zonal_z_Fields.append(updated_item)
                else:
                    updated_zonal_z_Fields.append(item)
            
            fields = ['Z_Max_1pct','Z_Mean_1pct',
                      'Z_Max_FVA00','Z_Mean_FVA00',
                      'Z_Max_FVA01','Z_Mean_FVA01',
                      'Z_Max_FVA02','Z_Mean_FVA02',
                      'Z_Max_FVA03','Z_Mean_FVA03']
            
            # Initialize an empty dictionary
            field_zonal_dict = {}

            # Build the dictionary pairing the lists fields with zonal_z_Fields
            for i in range(len(fields)):
                field_zonal_dict[fields[i]] = updated_zonal_z_Fields[i]

            arcpy.management.AddFields(
                in_table=x,
                field_description=[[fields[0],'DOUBLE',fields[0]],
                                [fields[1],'DOUBLE',fields[1]],
                                [fields[2],'DOUBLE',fields[2]],
                                [fields[3],'DOUBLE',fields[3]],
                                [fields[4],'DOUBLE',fields[4]],
                                [fields[5],'DOUBLE',fields[5]],
                                [fields[6],'DOUBLE',fields[6]],
                                [fields[7],'DOUBLE',fields[7]],
                                [fields[8],'DOUBLE',fields[8]],
                                [fields[9],'DOUBLE',fields[9]]])

            print('z fields added')

            for item in fields:
                for key, value in field_zonal_dict.items():
                    if key == item:
                        fieldsx = [key,value]
                        edit = arcpy.da.Editor(gdb)
                        edit.startEditing(False,True)
                        with arcpy.da.UpdateCursor(x,fieldsx) as cursor:
                            for row in cursor:
                                Z_max_value = row[1]
                                row[0] = Z_max_value
                                cursor.updateRow(row)
                        edit.stopEditing(True)
                print(f'"{fc_name}" {item} values updated')

            join_fc = arcpy.Describe(y)
            join_fc_name = join_fc.name
            product = arcpy.management.Dissolve(
                        in_features=x,
                        out_feature_class=z,
                        dissolve_field=[join_fc_name+'_FID',join_fc_name+"_Ranking"],
                        statistics_fields=[['Z_Max_1pct','MAX'],['Z_Mean_1pct','MEAN'],
                          ['Z_Max_FVA00','MAX'],['Z_Mean_FVA00','MEAN'],
                          ['Z_Max_FVA01','MAX'],['Z_Mean_FVA01','MEAN'],
                          ['Z_Max_FVA02','MAX'],['Z_Mean_FVA02','MEAN'],
                          ['Z_Max_FVA03','MAX'],['Z_Mean_FVA03','MEAN']])
            
            print('dissolve successful')    
            return product
        
        def exp_geo_point(u,v,w,x):

            fields = ['Z','Z_1pct','FVA00','FVA01','FVA02','FVA03']
            z_fields = ['Z_1pct','FVA00','FVA01','FVA02','FVA03']

             # Initialize an empty dictionary
            field_sur_dict = {}

            # Build the dictionary pairing the lists fields with zonal_z_Fields
            for i in range(len(z_fields)):
                field_sur_dict[z_fields[i]] = surface[i]

            arcpy.analysis.PairwiseIntersect(
                in_features=u,
                out_feature_class=v,
                join_attributes="ALL",
                output_type="INPUT")
            
            pt_fl = arcpy.management.MakeFeatureLayer(v,'pt_fl')

            arcpy.management.AddFields(
                in_table=v,
                field_description=[
                    [z_fields[0],'DOUBLE',z_fields[0]],
                    [z_fields[1],'DOUBLE',z_fields[1]],
                    [z_fields[2],'DOUBLE',z_fields[2]],
                    [z_fields[3],'DOUBLE',z_fields[3]],
                    [z_fields[4],'DOUBLE',z_fields[4]]])

            for item in z_fields:
                for key, value in field_sur_dict.items():
                    if key == item:
                        fields = [key,value]

                        AddSurfaceInformation(
                            in_feature_class=pt_fl,
                            in_surface=fields[1],
                            out_property="Z",
                            method="BILINEAR")

                        edit = arcpy.da.Editor(x)
                        edit.startEditing(False,True)
                        cursor_fields = ['Z',key]
                        with arcpy.da.UpdateCursor(pt_fl,cursor_fields) as cursor:
                            for row in cursor:
                                Z_value = row[0]
                                row[1] = Z_value
                                cursor.updateRow(row)
                        edit.stopEditing(True)
                    print(f'{fc_name} {key} values updated')

            product = arcpy.CopyFeatures_management(
                in_features=pt_fl,
                out_feature_class=w)

            return product
        
        if fc_name not in fc_dict:
            fc_dict[fc_name] = []
        fc_dict[fc_name].append(geo_type)
        print(fc_dict[fc_name])

        if "Point" in fc_dict[fc_name]:
            print(f'{fc_name} is a point layer :: adding surface information')
            arcpy.management.CalculateGeometryAttributes(
                in_features=fc,
                geometry_property=[['Lat','POINT_Y'],['Long','POINT_X']],
                coordinate_system=spa,
                coordinate_format='DD')
            with arcpy.EnvManager(workspace=dsp):
                exp_geo_point(int_list,geo_result_int,geo_result,gdb)
        else:
            with arcpy.EnvManager(workspace=dsp):
                exp_geo_line_poly(int_list,FID_Field,geo_result_join,geo_result_int,geo_result)

# clean up process data
for dsp in ds_paths:
    env(dsp)
    fc_list = arcpy.ListFeatureClasses()

    arcpy.SetProgressor("Step","Exporting Tables for Dataset: {0}...".format(dsp_final),0,len(fc_list))
    for fc in fc_list:
        fc_desc = arcpy.Describe(fc)
        fc_name = fc_desc.name
        fc_path = paths(dsp,fc_name)
        name_end = fc_name.split('_')[-1]

        if name_end == 'int' or name_end == 'join':
            arcpy.Delete_management(fc)
            print(f'{fc_name} is process feature :: fc deleted')
        else:
            print(f'{fc_name} is either final or original fc :: fc retained')

# clean up zonal tables
env(gdb)
tbl_list = arcpy.ListTables()
for tbl in tbl_list:
    tbl_desc = arcpy.Describe(tbl)
    tbl_name = tbl_desc.name
    tbl_path = paths(gdb,tbl_name)
    arcpy.Delete_management(tbl_path)
    if arcpy.Exists(tbl_path):
        print(f'{tbl} not deleted')
    else:
        print(f'{tbl} successfully deleted')

['C:\\Project_Files\\GIS\\Library\\Dev_Apps\\HMP_Tools\\HMP_Tools\\Temp_Processing.gdb\\Natural_cultural_historical_resource']
5
C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb\Natural_cultural_historical_resource\St_Cloud_Park_Building
['Point']
St_Cloud_Park_Building is a point layer :: adding surface information
St_Cloud_Park_Building Z_1pct values updated
St_Cloud_Park_Building FVA00 values updated
St_Cloud_Park_Building FVA01 values updated
St_Cloud_Park_Building FVA02 values updated
St_Cloud_Park_Building FVA03 values updated
St_Cloud_Park_Building Z_1pct values updated
St_Cloud_Park_Building FVA00 values updated
St_Cloud_Park_Building FVA01 values updated
St_Cloud_Park_Building FVA02 values updated
St_Cloud_Park_Building FVA03 values updated
St_Cloud_Park_Building Z_1pct values updated
St_Cloud_Park_Building FVA00 values updated
St_Cloud_Park_Building FVA01 values updated
St_Cloud_Park_Building FVA02 values updated
St_Cloud_Park_Building FVA03 value

In [ ]:
statistics_fields=[['Z_Max_1pct','MAX'],['Z_Mean_1pct','MEAN'],
                          ['Z_Max_FVA00','MAX'],['Z_Mean_FVA00','MEAN'],
                          ['Z_Max_FVA01','MAX'],['Z_Mean_FVA01','MEAN'],
                          ['Z_Max_FVA02','MAX'],['Z_Mean_FVA02','MEAN'],
                          ['Z_Max_FVA03','MAX'],['Z_Mean_FVA03','MEAN']])

In [ ]:
import os,sys, arcpy
import pandas as pd
import numpy as np
from arcpy.sa import* 

arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

gdb = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb'
SFHA = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\FFRMS_Processing\Working_Data\Eff_data.gdb\S_FLD_HAZ_AR'
Depth_Grid = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\FFRMS_Processing\Production\County_#####\ProcessedData\WSEL\FL_12097_17N_RIV_03m_WSEL.tif'
FVA_00 = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\FFRMS_Processing\Working_Data\RIV_data.gdb\freeboard0'
FVA_01 = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\FFRMS_Processing\Working_Data\RIV_data.gdb\freeboard1'
FVA_02 = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\FFRMS_Processing\Working_Data\RIV_data.gdb\freeboard2'
FVA_03 = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\FFRMS_Processing\Working_Data\RIV_data.gdb\freeboard3'
csv_point = r'C:\Project_Files\GIS\HMP+\St_Cloud+\2.0 Tables\1.0 Working\Sensitivty_Analysis\JPA_Sensitivy_FVA_Analysis_Point.csv'
csv_ln_pt = r'C:\Project_Files\GIS\HMP+\St_Cloud+\2.0 Tables\1.0 Working\Sensitivty_Analysis\St_Cloud_Sensitivty_Analysis_Line_Poly.csv'

datasets = []
fc_all = []
ds_paths = []
fc_dict = {}
surface = [Depth_Grid,FVA_00,FVA_01,FVA_02,FVA_03]
fields = ['Z','Z_1pct','FVA00','FVA01','FVA02','FVA03']
z_fields = ['Z_1pct','FVA00','FVA01','FVA02','FVA03']

 # Initialize an empty dictionary
field_sur_dict = {}

# Build the dictionary pairing the lists fields with zonal_z_Fields
for i in range(len(z_fields)):
    field_sur_dict[z_fields[i]] = surface[i]

def env(x):
    arcpy.env.workspace = x
    return arcpy.env.workspace

def paths(x,y):
    path = os.path.join(x,y)
    return path

env(gdb)

feature_datasets = arcpy.ListDatasets()
for dataset in feature_datasets:
    datasets.append(dataset)

for ds in datasets:
    ds_path = paths(gdb,ds)
    ds_paths.append(ds_path)
print(ds_paths)

for dsp in ds_paths:
    env(dsp)
    fc_list = arcpy.ListFeatureClasses()
    print('Total feature count in feature dataset {0} :: {1}'.format(dsp.split("\\")[-1],len(fc_list)))
    parse_dsp = dsp.split('\\')
    dsp_name = parse_dsp[-1]
    dsp_final = dsp_name.replace("_"," ")

    arcpy.SetProgressor("Step","Exporting Tables for Dataset: {0}...".format(dsp_final),0,len(fc_list))
    for fc in fc_list:
        fc_desc = arcpy.Describe(fc)
        spa = fc_desc.SpatialReference
        print(spa)
        spa_name = spa.name
        print(spa_name)
        fc_name = fc_desc.name
        fc_path = paths(dsp,fc_name)
        print(fc_path)
        geo_type = fc_desc.shapeType
        int_list = [SFHA,fc]
        geo_result_int = fc_path+"_int"
        geo_result = fc_path+'_final'

        def exp_geo_point(u,v,w,x):

            arcpy.analysis.PairwiseIntersect(
                in_features=u,
                out_feature_class=v,
                join_attributes="ALL",
                output_type="INPUT")
            
            pt_fl = arcpy.management.MakeFeatureLayer(v,'pt_fl')

            arcpy.management.AddFields(
                in_table=v,
                field_description=[
                    [z_fields[0],'DOUBLE',z_fields[0]],
                    [z_fields[1],'DOUBLE',z_fields[1]],
                    [z_fields[2],'DOUBLE',z_fields[2]],
                    [z_fields[3],'DOUBLE',z_fields[3]],
                    [z_fields[4],'DOUBLE',z_fields[4]]])

            for item in z_fields:
                for key, value in field_sur_dict.items():
                    if key == item:
                        fields = [key,value]

                        AddSurfaceInformation(
                            in_feature_class=pt_fl,
                            in_surface=fields[1],
                            out_property="Z",
                            method="BILINEAR")

                        edit = arcpy.da.Editor(x)
                        edit.startEditing(False,True)
                        cursor_fields = ['Z',key]
                        with arcpy.da.UpdateCursor(pt_fl,cursor_fields) as cursor:
                            for row in cursor:
                                Z_value = row[0]
                                row[1] = Z_value
                                cursor.updateRow(row)
                        edit.stopEditing(True)
                    print(f'{fc_name} {key} values updated')

            product = arcpy.CopyFeatures_management(
                in_features=pt_fl,
                out_feature_class=w)

            return product

        if fc_name not in fc_dict:
            fc_dict[fc_name] = []
        fc_dict[fc_name].append(geo_type)
        print(fc_dict[fc_name])

        if "Point" in fc_dict[fc_name]:
            print(f'{fc_name} is a point layer :: adding surface information')
            with arcpy.EnvManager(workspace=dsp):
                exp_geo_point(int_list,geo_result_int,geo_result,gdb)

        else:
            print(f'{fc_name} is not a point layer :: no surface information added')

# clean up process data
for dsp in ds_paths:
    env(dsp)
    fc_list = arcpy.ListFeatureClasses()

    arcpy.SetProgressor("Step","Exporting Tables for Dataset: {0}...".format(dsp_final),0,len(fc_list))
    for fc in fc_list:
        fc_desc = arcpy.Describe(fc)
        fc_name = fc_desc.name
        fc_path = paths(dsp,fc_name)
        name_end = fc_name.split('_')[-1]

        if name_end == 'int' or name_end == 'join':
            arcpy.Delete_management(fc)
            print(f'{fc_name} is process feature :: fc deleted')
        else:
            print(f'{fc_name} is either final or original fc :: fc retained')

# clean up zonal tables
env(gdb)
tbl_list = arcpy.ListTables()
for tbl in tbl_list:
    tbl_desc = arcpy.Describe(tbl)
    tbl_name = tbl_desc.name
    tbl_path = paths(gdb,tbl_name)
    arcpy.Delete_management(tbl_path)
    if arcpy.Exists(tbl_path):
        print(f'{tbl} not deleted')
    else:
        print(f'{tbl} successfully deleted')




['C:\\Project_Files\\GIS\\Library\\Dev_Apps\\HMP_Tools\\HMP_Tools\\Temp_Processing.gdb\\Natural_cultural_historical_resource']
Total feature count in feature dataset Natural_cultural_historical_resource :: 5
<geoprocessing spatial reference object object at 0x000002E4292502F0>
NAD_1983_StatePlane_Florida_East_FIPS_0901_Feet
C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb\Natural_cultural_historical_resource\St_Cloud_Park_Building
['Point']
St_Cloud_Park_Building is a point layer :: adding surface information
St_Cloud_Park_Building Z_1pct values updated
St_Cloud_Park_Building FVA00 values updated
St_Cloud_Park_Building FVA01 values updated
St_Cloud_Park_Building FVA02 values updated
St_Cloud_Park_Building FVA03 values updated
St_Cloud_Park_Building Z_1pct values updated
St_Cloud_Park_Building FVA00 values updated
St_Cloud_Park_Building FVA01 values updated
St_Cloud_Park_Building FVA02 values updated
St_Cloud_Park_Building FVA03 values updated
St_Cloud_Park_

tables export breaking it down

In [ ]:
import os,sys, arcpy
import pandas as pd
import numpy as np
from arcpy.sa import* 

arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

fc = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb\Natural_cultural_historical_resource\fire_stations_final'
csv_point = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Folder\JPA_Sensitivity_Analysis_Point.csv'
csv_ln_pt = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Folder\JPA_Sensitivity_Analysis_Line_Poly.csv'
csv_asset = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Folder\JPA_Sensitivity_Analysis_Asset_List.csv'
dsp_name = 'Natural_cultural_historical_resource'
fc_desc = arcpy.Describe(fc)
fc_name = fc_desc.name
og_name = fc_name.replace('_final','')
FID_Field = 'FID_'+og_name
print(FID_Field)


field_names = [field.name for field in arcpy.ListFields(fc)]
print(field_names)


def df_exp_asset(w,x,z):

    fields = ['Ranking','Asset','Name','Lat','Long','Z_1pct','FVA00','FVA01','FVA02','FVA03']

    # Reading feature class data into a pandas DataFrame
    data = [row for row in arcpy.da.SearchCursor(w, x)]
    df = pd.DataFrame(data, columns=x)

    # Find the column name that matches 'Name' in a case-insensitive way
    reference_column = next((col for col in df.columns if col.lower() == 'name'), None)

    if reference_column is not None:
        # Rename the found column to 'Name'
        df.rename(columns={reference_column: 'Name'}, inplace=True)

    print(df.head())

    filtered_df = df[fields].copy()
    filtered_df.loc[:,'Asset_Type'] = fc_name
    filtered_df.loc[:,'Asset_Group'] = dsp_name

    filtered_df['Sensitivity'] = filtered_df['Ranking'].apply(lambda x: 'Low' if x == 1 else ('Med' if x == 2 else 'High'))
    print(filtered_df.head())

    out_table = filtered_df.to_csv(z, mode='a', header=False, index=False)
    print("CSV Successfully Appended")

    return out_table



df_exp_asset(fc,field_names,csv_asset)


FID_fire_stations
['OBJECTID', 'Shape', 'FID_S_FLD_HAZ_AR', 'DFIRM_ID', 'VERSION_ID', 'FLD_AR_ID', 'STUDY_TYP', 'FLD_ZONE', 'ZONE_SUBTY', 'SFHA_TF', 'STATIC_BFE', 'V_DATUM', 'DEPTH', 'LEN_UNIT', 'VELOCITY', 'VEL_UNIT', 'AR_REVERT', 'AR_SUBTRV', 'BFE_REVERT', 'DEP_REVERT', 'DUAL_ZONE', 'SOURCE_CIT', 'Ranking', 'FID_fire_stations', 'OBJECTID_1', 'NAME', 'HAZUSID', 'ADDRESS', 'CITY', 'ZIP', 'COUNTY', 'X', 'Y', 'LONGLAT', 'USNG', 'OWNERSHIP', 'PARCELID', 'FACILITY_T', 'EFCLASS', 'AREA_SQ_FT', 'BACKUP_POW', 'KITCHENS', 'REPLACEMEN', 'EQ_DESIGN_', 'EQ_BUILD_T', 'EQ_DF_TYPE', 'LD_SUS', 'LIQU_SUS', 'SOIL_TYPE', 'WATER_DEPT', 'STRUC_DAMA', 'FLOOD_BLDG', 'CONTENT_DA', 'FIRST_FLOO', 'FLOOD_RETU', 'FLOOD_STR_', 'EntityName', 'Asset_Elev', 'Own_Maintn', 'Asset', 'Asset_Type', 'Asset_Group', 'Lat', 'Long', 'Z_1pct', 'FVA00', 'FVA01', 'FVA02', 'FVA03', 'Z']
   OBJECTID                                    Shape  FID_S_FLD_HAZ_AR  \
0         1  (553976.4829300791, 1414580.9225168228)                31 

In [ ]:
import os,sys, arcpy
import pandas as pd
import numpy as np
from arcpy.sa import* 

arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

gdb = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb'
csv_point = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Folder\JPA_Sensitivity_Analysis_Point.csv'
csv_ln_pt = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Folder\JPA_Sensitivity_Analysis_Line_Poly.csv'

datasets = []
fc_all = []
ds_paths = []

def env(x):
    arcpy.env.workspace = x
    return arcpy.env.workspace

def paths(x,y):
    path = os.path.join(x,y)
    return path

env(gdb)

feature_datasets = arcpy.ListDatasets()
for dataset in feature_datasets:
    datasets.append(dataset)

for ds in datasets:
    ds_path = paths(gdb,ds)
    ds_paths.append(ds_path)
print(ds_paths)

# List to hold field names

def df_exp_line_poly(v,w,x,y,z):

    out_field_list = [
        'MAX_Z_Max_1pct',
        'MEAN_Z_Mean_1pct',
        'MAX_Z_Max_FVA00',
        'MEAN_Z_Mean_FVA00',
        'MAX_Z_Max_FVA01',
        'MEAN_Z_Mean_FVA01',
        'MAX_Z_Max_FVA02',
        'MEAN_Z_Mean_FVA02',
        'MAX_Z_Max_FVA03',
        'MEAN_Z_Mean_FVA03']

    # Reading feature class data into a pandas DataFrame
    data = [row for row in arcpy.da.SearchCursor(w, x)]
    df = pd.DataFrame(data, columns=x)

    # Displaying the DataFrame
    print(df.head())  # Displaying the first few rows of the DataFrame

    # Handling <Null> values (assuming they are represented as NaN)
    df[out_field_list[0]].fillna(np.nan, inplace=True)
    df[out_field_list[1]].fillna(np.nan, inplace=True)
    df[out_field_list[2]].fillna(np.nan, inplace=True)
    df[out_field_list[3]].fillna(np.nan, inplace=True)
    df[out_field_list[4]].fillna(np.nan, inplace=True)
    df[out_field_list[5]].fillna(np.nan, inplace=True)
    df[out_field_list[6]].fillna(np.nan, inplace=True)
    df[out_field_list[7]].fillna(np.nan, inplace=True)
    df[out_field_list[8]].fillna(np.nan, inplace=True)
    df[out_field_list[9]].fillna(np.nan, inplace=True)

    # Finding the row with the maximum ranking for each OID
    result = df.loc[df.groupby(y)[v].idxmax()]

    # Creating a new DataFrame with the results
    new_df = result[
        [y,
         v,
        out_field_list[0],
        out_field_list[1],
        out_field_list[2],
        out_field_list[3],
        out_field_list[4],
        out_field_list[5],
        out_field_list[6],
        out_field_list[7],
        out_field_list[8],
        out_field_list[9]]].reset_index(drop=True)

    # Displaying the new DataFrame
    num_rows=len(new_df)
    print(num_rows)
    new_df.head() 

    # Calculate frequency of each unique Ranking in the entire DataFrame 'new_df'
    frequency = new_df[v].value_counts()

    # Check the frequency counts and head of the DataFrame for diagnostics
    print("Frequency counts:\n", frequency.head())

    # Calculate the maximum MAX_Z_Max and average MEAN_Z_Mean for each 'Ranking'
    aggregated_data = new_df.groupby(v).agg({
        out_field_list[0]: 'max',
        out_field_list[1]: 'mean',
        out_field_list[2]: 'max',
        out_field_list[3]: 'mean',
        out_field_list[4]: 'max',
        out_field_list[5]: 'mean',
        out_field_list[6]: 'max',
        out_field_list[7]: 'mean',
        out_field_list[8]: 'max',
        out_field_list[9]: 'mean'
    }).reset_index()

    # Check the aggregated data for diagnostics
    print("Aggregated data:\n", aggregated_data.head())

    # Create a DataFrame with specified columns ('Ranking', 'Frequency')
    new1_df = pd.DataFrame({
        v: frequency.index,
        'Frequency': frequency.values,
    })

    # Merge 'new_df' with 'aggregated_data' based on 'Ranking' to get 'Max_Z_Max' and 'MEAN_Z_Mean'
    new2_df = new1_df.merge(aggregated_data, on=v)
    new2_df['Asset_Type'] = fc_name
    new2_df['Asset_Group'] =dsp_name

    # Displaying the new DataFrame 'new2_df'
    print(new2_df.head())

    # Mapping rankings to sensitivity levels using lambda function
    new2_df['Sensitivity'] = new2_df[v].apply(lambda x: 'Low' if x == 1 else ('Med' if x == 2 else 'High'))

    out_table = new2_df.to_csv(z, mode='a', header=False, index=False)
    print("CSV Successfully Appended")

    return out_table

def df_exp_point(w,x,y,z):

    z_fields = ['Z_1pct','FVA00','FVA01','FVA02','FVA03']

    # Reading feature class data into a pandas DataFrame
    data = [row for row in arcpy.da.SearchCursor(w, x)]
    df = pd.DataFrame(data, columns=x)

    # Displaying the DataFrame
    print(df.head())  # Displaying the first few rows of the DataFrame

    # Handling <Null> values (assuming they are represented as NaN)
    df['Z'].fillna(np.nan, inplace=True)

    # Finding the row with the maximum ranking for each OID
    result = df.loc[df.groupby(y)['Ranking'].idxmax()]

    # Creating a new DataFrame with the results
    new_df = result[[
        y,
        'Ranking',
        z_fields[0],
        z_fields[1],
        z_fields[2],
        z_fields[3],
        z_fields[4]]].reset_index(drop=True)

    # Displaying the new DataFrame
    num_rows=len(new_df)
    print(num_rows)
    new_df.head() 

    # Calculate frequency of each unique Ranking in the entire DataFrame 'new_df'
    frequency = new_df['Ranking'].value_counts()

    # Check the frequency counts and head of the DataFrame for diagnostics
    print("Frequency counts:\n", frequency.head())

    # Calculate the maximum MAX_Z_Max and average MEAN_Z_Mean for each 'Ranking'
    aggregated_data = new_df.groupby('Ranking').agg({
        z_fields[0]: 'max',
        z_fields[1]: 'max',
        z_fields[2]: 'max',
        z_fields[3]: 'max',
        z_fields[4]: 'max',
    }).reset_index()

    # Check the aggregated data for diagnostics
    print("Aggregated data:\n", aggregated_data.head())

    # Create a DataFrame with specified columns ('Ranking', 'Frequency')
    new1_df = pd.DataFrame({
        'Ranking': frequency.index,
        'Frequency': frequency.values,
    })

    # Merge 'new_df' with 'aggregated_data' based on 'Ranking' to get 'Max_Z_Max'
    new2_df = new1_df.merge(aggregated_data, on='Ranking')
    new2_df['Asset_Type'] = fc_name
    new2_df['Asset_Group'] =dsp_name

    # Displaying the new DataFrame 'new2_df'
    print(new2_df.head())

    # Mapping rankings to sensitivity levels using lambda function
    new2_df['Sensitivity'] = new2_df['Ranking'].apply(lambda x: 'Low' if x == 1 else ('Med' if x == 2 else 'High'))

    out_table = new2_df.to_csv(z, mode='a', header=False, index=False)
    print("CSV Successfully Appended")

    return out_table

for dsp in ds_paths:
    env(dsp)
    fc_list = arcpy.ListFeatureClasses()
    print('Total feature count in feature dataset {0} :: {1}'.format(dsp.split("\\")[-1],len(fc_list)))
    parse_dsp = dsp.split('\\')
    dsp_name = parse_dsp[-1]
    dsp_final = dsp_name.replace("_"," ")

    arcpy.SetProgressor("Step","Exporting Tables for Dataset: {0}...".format(dsp_final),0,len(fc_list))
    for fc in fc_list:
        fc_desc = arcpy.Describe(fc)
        fc_name = fc_desc.name
        print(fc_name)
        geo_type = fc_desc.shapeType

        if fc_name.split('_')[-1] == 'final':
            if geo_type == "Point":
                print('is point feature')
                og_name = fc_name.replace('_final','')
                FID_Field = 'FID_'+og_name
                field_names = [field.name for field in arcpy.ListFields(fc)]
                df_exp_point(fc,field_names,FID_Field,csv_point)
            elif geo_type == "Polyline":
                print('is polyline feature')
                og_name = fc_name.replace('_final','')
                FID_Field = og_name+'_int_FID'
                Ranking = og_name+'_int_Ranking'
                field_names = [field.name for field in arcpy.ListFields(fc)]
                df_exp_line_poly(Ranking,fc,field_names,FID_Field,csv_ln_pt)
            elif geo_type == "Polygon":
                print('is polygon feature')
                og_name = fc_name.replace('_final','')
                FID_Field = og_name+'_int_FID'
                Ranking = og_name+'_int_Ranking'
                field_names = [field.name for field in arcpy.ListFields(fc)]
                df_exp_line_poly(Ranking,fc,field_names,FID_Field,csv_ln_pt)

['C:\\Project_Files\\GIS\\Library\\Dev_Apps\\HMP_Tools\\HMP_Tools\\Temp_Processing.gdb\\Natural_cultural_historical_resource']
Total feature count in feature dataset Natural_cultural_historical_resource :: 6
St_Cloud_Park_Building
St_Cloud_Park_ConnectivityLines
St_Cloud_Park_Park_Boundaries
St_Cloud_Park_ConnectivityLines_final
is polyline feature
   OBJECTID                                    Shape  \
0         1   (565883.5107871166, 1411527.684771993)   
1         2  (557540.3261825906, 1419695.0628443253)   
2         3     (557553.698725293, 1419668.66944381)   
3         4  (558991.2337671384, 1419679.0259869434)   
4         5  (561632.3456408223, 1425064.1858570005)   

   St_Cloud_Park_ConnectivityLines_int_Ranking  \
0                                          1.0   
1                                          1.0   
2                                          6.0   
3                                          1.0   
4                                          1.0   

  St_Cloud_

calc lat and long for allpoint features

In [ ]:
import os,sys, arcpy
import pandas as pd
import numpy as np
from arcpy.sa import* 

arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

gdb = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\Florida\Critical_Asset\Boundary_Redo\County_no_city_Assets.gdb'
SFHA = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb\County_S_FLD_HAZ_AR'
Depth_Grid = r'C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\USGS\Lidar\Data_Download\Derived_DEM\Depth_Con.tif'
csv_point = r'C:\Project_Files\GIS\HMP+\St_Cloud+\2.0 Tables\1.0 Working\Sensitivty_Analysis\St_Cloud_Sensitivty_Analysis_Point.csv'
csv_ln_pt = r'C:\Project_Files\GIS\HMP+\St_Cloud+\2.0 Tables\1.0 Working\Sensitivty_Analysis\St_Cloud_Sensitivty_Analysis_Line_Poly.csv'

sfha = arcpy.GetParameterAsText(0)

datasets = []
fc_all = []
ds_paths = []
fc_dict = {}

def env(x):
    arcpy.env.workspace = x
    return arcpy.env.workspace

def paths(x,y):
    path = os.path.join(x,y)
    return path

env(gdb)

feature_datasets = arcpy.ListDatasets()
for dataset in feature_datasets:
    datasets.append(dataset)

for ds in datasets:
    ds_path = paths(gdb,ds)
    ds_paths.append(ds_path)
print(ds_paths)


for dsp in ds_paths:
    env(dsp)
    fc_list = arcpy.ListFeatureClasses()
    print(len(fc_list))
    parse_dsp = dsp.split('\\')
    dsp_name = parse_dsp[-1]
    dsp_final = dsp_name.replace("_"," ")

    arcpy.SetProgressor("Step","Exporting Tables for Dataset: {0}...".format(dsp_final),0,len(fc_list))
    for fc in fc_list:
        fc_desc = arcpy.Describe(fc)
        spa = fc_desc.SpatialReference
        print(spa)
        spa_name = spa.name
        print(spa_name)
        fc_name = fc_desc.name
        fc_path = paths(dsp,fc_name)
        print(fc_path)
        geo_type = fc_desc.shapeType


        if geo_type == 'Point':
            print(f'{fc} is point type')
            arcpy.management.CalculateGeometryAttributes(
                in_features=fc,
                geometry_property=[['Lat','POINT_Y'],['Long','POINT_X']],
                coordinate_system=spa,
                coordinate_format='DD')
            print('lat and long successfully calculated')
        else:
            print(f'{fc} is {geo_type} type  :: no geometry claculated')
            
        arcpy.SetProgressorLabel("Exporting {0} data...".format(fc_name.replace("_"," ")))

['C:\\Project_Files\\GIS\\HMP+\\St_Cloud+\\4.0 Reference Data\\Florida\\Critical_Asset\\Boundary_Redo\\County_no_city_Assets.gdb\\Critical_community_emergency_facilities', 'C:\\Project_Files\\GIS\\HMP+\\St_Cloud+\\4.0 Reference Data\\Florida\\Critical_Asset\\Boundary_Redo\\County_no_city_Assets.gdb\\Critical_infrastructure', 'C:\\Project_Files\\GIS\\HMP+\\St_Cloud+\\4.0 Reference Data\\Florida\\Critical_Asset\\Boundary_Redo\\County_no_city_Assets.gdb\\Natural_cultural_historical_resource', 'C:\\Project_Files\\GIS\\HMP+\\St_Cloud+\\4.0 Reference Data\\Florida\\Critical_Asset\\Boundary_Redo\\County_no_city_Assets.gdb\\Transportation_and_evacuation_route']
18
<geoprocessing spatial reference object object at 0x000002A5A6C35CD0>
NAD_1983_StatePlane_Florida_East_FIPS_0901_Feet
C:\Project_Files\GIS\HMP+\St_Cloud+\4.0 Reference Data\Florida\Critical_Asset\Boundary_Redo\County_no_city_Assets.gdb\Critical_community_emergency_facilities\emergency_operation_centers
emergency_operation_centers is 

In [ ]:
import arcpy, os, sys
import pandas as pd
arcpy.env.overwriteOutput = True

arcpy.SetProgressor("default","Initiating VA Tab Export Tool...")

gdb = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\StCloud_Cristical_Assets_Filtered_Assets.gdb'
out_table = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Folder\Test.xlsx'

combined_df = pd.DataFrame()
excel_writer = pd.ExcelWriter(out_table)
datasets = []
fc_all = []
ds_paths = []
fc_dict = {}

def env(x):
    arcpy.env.workspace = x
    return arcpy.env.workspace

def paths(x,y):
    path = os.path.join(x,y)
    return path

env(gdb)

feature_datasets = arcpy.ListDatasets()
for dataset in feature_datasets:
    datasets.append(dataset)

for ds in datasets:
    ds_path = paths(gdb,ds)
    ds_paths.append(ds_path)
print(ds_paths)


for dsp in ds_paths:
    env(dsp)
    fc_list = arcpy.ListFeatureClasses()
    parse_dsp = dsp.split('\\')
    dsp_name = parse_dsp[-1]
    dsp_final = dsp_name.replace("_"," ")
    for fc in fc_list:
        desc = arcpy.Describe(fc)
        fc_name = desc.name
        geo_type = desc.shapeType
        fc_path = os.path.join(dsp,fc_name)
        if fc_name not in fc_dict:
            fc_dict[fc_name] = []
        fc_dict[fc_name].append(geo_type)

print(fc_dict['affordable_public_housing'])

for fc_name, geo_types in fc_dict.items():
    print(f"Feature Class: {fc_name}, Geometry Types: {', '.join(geo_types)}")

['C:\\Project_Files\\GIS\\Library\\Dev_Apps\\HMP_Tools\\HMP_Tools\\StCloud_Cristical_Assets_Filtered_Assets.gdb\\Critical_community_emergency_facilities', 'C:\\Project_Files\\GIS\\Library\\Dev_Apps\\HMP_Tools\\HMP_Tools\\StCloud_Cristical_Assets_Filtered_Assets.gdb\\Critical_infrastructure', 'C:\\Project_Files\\GIS\\Library\\Dev_Apps\\HMP_Tools\\HMP_Tools\\StCloud_Cristical_Assets_Filtered_Assets.gdb\\Natural_cultural_historical_resource', 'C:\\Project_Files\\GIS\\Library\\Dev_Apps\\HMP_Tools\\HMP_Tools\\StCloud_Cristical_Assets_Filtered_Assets.gdb\\Transportation_and_evacuation_route']
['Point']
Feature Class: affordable_public_housing, Geometry Types: Point
Feature Class: disaster_recovery_centers, Geometry Types: Point
Feature Class: fire_stations, Geometry Types: Point
Feature Class: hospitals, Geometry Types: Point
Feature Class: law_enforcement_facilities, Geometry Types: Point
Feature Class: local_government_facilities, Geometry Types: Point
Feature Class: risk_shelter_inventory

use dictionary to recall the geotyp

bring in bin csv to help create df

build functions to geoprocess

creat df from derived layer and manipulate it

In [ ]:
import os,sys, arcpy
import pandas as pd
import numpy as np

gdb = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb'
fc = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb\surface_waters_flowlines_int_dis_test'
csv = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Folder\Test.csv'
fc = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Processing.gdb\surface_waters_flowlines_int_dis_test'

desc = arcpy.Describe(fc)
fc_nm = desc.name

# List to hold field names
field_names = [field.name for field in arcpy.ListFields(fc)]

def df_exp(w,x,y,z):

    # Reading feature class data into a pandas DataFrame
    data = [row for row in arcpy.da.SearchCursor(w, x)]
    df = pd.DataFrame(data, columns=x)

    # Displaying the DataFrame
    print(df.head())  # Displaying the first few rows of the DataFrame

    # Handling <Null> values (assuming they are represented as NaN)
    df['MAX_Z_Max'].fillna(np.nan, inplace=True)
    df['MEAN_Z_Mean'].fillna(np.nan, inplace=True)

    # Finding the row with the maximum ranking for each OID
    result = df.loc[df.groupby(y)['Ranking'].idxmax()]

    # Creating a new DataFrame with the results
    new_df = result[[y, 'Ranking', 'MAX_Z_Max','MEAN_Z_Mean']].reset_index(drop=True)

    # Displaying the new DataFrame
    num_rows=len(new_df)
    print(num_rows)
    new_df.head() 

    # Calculate frequency of each unique Ranking in the entire DataFrame 'new_df'
    frequency = new_df['Ranking'].value_counts()

    # Check the frequency counts and head of the DataFrame for diagnostics
    print("Frequency counts:\n", frequency.head())

    # Calculate the maximum MAX_Z_Max and average MEAN_Z_Mean for each 'Ranking'
    aggregated_data = new_df.groupby('Ranking').agg({
        'MAX_Z_Max': 'max',
        'MEAN_Z_Mean': 'mean'
    }).reset_index()

    # Check the aggregated data for diagnostics
    print("Aggregated data:\n", aggregated_data.head())

    # Create a DataFrame with specified columns ('Ranking', 'Frequency')
    new1_df = pd.DataFrame({
        'Ranking': frequency.index,
        'Frequency': frequency.values,
    })

    # Merge 'new_df' with 'aggregated_data' based on 'Ranking' to get 'Max_Z_Max' and 'MEAN_Z_Mean'
    new2_df = new1_df.merge(aggregated_data, on='Ranking')
    new2_df['Asset_Type'] = fc_nm
    new2_df['Asset_Group'] = "Test_Group"

    # Displaying the new DataFrame 'new2_df'
    print(new2_df.head())

    out_table = new2_df.to_csv(z, mode='a', header=False, index=False)
    print("CSV Successfully Appended")

    return out_table

df_exp(fc,field_names,FID_Field,csv)

   OBJECTID                                    Shape  \
0         1   (591634.1210868994, 1426900.470957766)   
1         2  (554551.8603882454, 1411195.0267295726)   
2         3  (589259.2041952033, 1433880.6654133238)   
3         4  (574411.5735048054, 1424090.4179453372)   
4         5  (574417.0426066201, 1424702.9060457833)   

   FID_surface_waters_flowlines  Ranking  MAX_Z_Max  MEAN_Z_Mean  Shape_Length  
0                             1      3.0   3.005693     2.364928    455.595912  
1                             2      1.0        NaN          NaN    280.276982  
2                             3      4.0        NaN          NaN    186.395357  
3                             4      1.0        NaN          NaN   1128.342101  
4                             4      2.0   4.800204     0.000000     96.683656  
291
Frequency counts:
 5.0    95
1.0    66
4.0    66
6.0    52
2.0     8
Name: Ranking, dtype: int64
Aggregated data:
    Ranking  MAX_Z_Max  MEAN_Z_Mean
0      1.0        NaN  

analyze df and generate new df that returns the z values of the highest ranked OID

In [ ]:
import os,sys, arcpy
import pandas as pd
import numpy as np

# Handling <Null> values (assuming they are represented as NaN)
df['MAX_Z_Max'].fillna(np.nan, inplace=True)
df['MEAN_Z_Mean'].fillna(np.nan, inplace=True)
# Finding the row with the maximum ranking for each OID
result = df.loc[df.groupby(FID_Field)['Ranking'].idxmax()]

# Creating a new DataFrame with the results
new_df = result[[FID_Field, 'Ranking', 'MAX_Z_Max','MEAN_Z_Mean']].reset_index(drop=True)

# Displaying the new DataFrame
num_rows=len(new_df)
print(num_rows)
new_df.head() 

291


,FID_surface_waters_flowlines,Ranking,MAX_Z_Max,MEAN_Z_Mean
0,1,3.0,3.005693,2.364928
1,2,1.0,NaN,NaN
2,3,4.0,NaN,NaN
3,4,5.0,5.171135,4.637860
4,5,6.0,4.295323,3.428716


use the new df to create the final df to export

In [ ]:
import os,sys, arcpy
import pandas as pd
import numpy as np

# Assuming 'new_df' contains your data

# Calculate frequency of each unique Ranking in the entire DataFrame 'new_df'
frequency = new_df['Ranking'].value_counts()

# Check the frequency counts and head of the DataFrame for diagnostics
print("Frequency counts:\n", frequency.head())

# Calculate the maximum MAX_Z_Max and average MEAN_Z_Mean for each 'Ranking'
aggregated_data = new_df.groupby('Ranking').agg({
    'MAX_Z_Max': 'max',
    'MEAN_Z_Mean': 'mean'
}).reset_index()

# Check the aggregated data for diagnostics
print("Aggregated data:\n", aggregated_data.head())

# Create a DataFrame with specified columns ('Ranking', 'Frequency')
new1_df = pd.DataFrame({
    'Ranking': frequency.index,
    'Frequency': frequency.values,
})

# Merge 'new_df' with 'aggregated_data' based on 'Ranking' to get 'Max_Z_Max' and 'MEAN_Z_Mean'
new2_df = new1_df.merge(aggregated_data, on='Ranking')
new2_df['Asset_Type'] = fc_nm
new2_df['Asset_Group'] = "Test_Group"

# Displaying the new DataFrame 'new2_df'
print(new2_df.head())

csv = r'C:\Project_Files\GIS\Library\Dev_Apps\HMP_Tools\HMP_Tools\Temp_Folder\Test.csv'

new2_df.to_csv(csv, mode='a', header=False, index=False)
print("CSV Successfully Appended")

Frequency counts:
 5.0    95
1.0    66
4.0    66
6.0    52
2.0     8
Name: Ranking, dtype: int64
Aggregated data:
    Ranking  MAX_Z_Max  MEAN_Z_Mean
0      1.0        NaN          NaN
1      2.0   0.676682     0.676682
2      3.0   3.065215     2.438675
3      4.0        NaN          NaN
4      5.0   7.226077     2.149353
   Ranking  Frequency  MAX_Z_Max  MEAN_Z_Mean  \
0      5.0         95   7.226077     2.149353   
1      1.0         66        NaN          NaN   
2      4.0         66        NaN          NaN   
3      6.0         52   9.352051     4.128429   
4      2.0          8   0.676682     0.676682   

                              Asset_Type Asset_Group  
0  surface_waters_flowlines_int_dis_test  Test_Group  
1  surface_waters_flowlines_int_dis_test  Test_Group  
2  surface_waters_flowlines_int_dis_test  Test_Group  
3  surface_waters_flowlines_int_dis_test  Test_Group  
4  surface_waters_flowlines_int_dis_test  Test_Group  
CSV Successfully Appended


pivot the df based on ranking

In [ ]:
# Assuming 'new_df' contains your data

# Resetting the index to have a numerical index as 'index'
new2_df.reset_index(drop=True, inplace=True)

# Creating a pivot table to reshape the data based on 'Ranking' values
pivot_table = new2_df.pivot(columns='Ranking')

# Flattening the pivot table columns and resetting the column names
pivot_table.columns = [f'{col[0]}_{col[1]}' for col in pivot_table.columns.values]

# Initialize a DataFrame to populate with flattened columns
flattened_df = pd.DataFrame(columns=pivot_table.columns)

# Iterate through each unique 'Ranking' value in new_df and fill the flattened DataFrame
for ranking in new2_df['Ranking'].unique():
    rows_for_ranking = new2_df[new2_df['Ranking'] == ranking]
    for column in pivot_table.columns:
        flattened_df.loc[0, column] = rows_for_ranking[column.split('_')[0] + '_' + str(ranking)].values[0]

# Displaying the populated flattened DataFrame
print(flattened_df)


KeyError: 'Frequency_5.0'